<a href="https://colab.research.google.com/github/SIBI-MS/ML-Projects/blob/main/FinetuningLlama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install accelerate peft transformers bitsandbytes

In [5]:
!pip install trl


In [6]:
import os
import torch
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import(
    LoraConfig,
    PeftModel,
)
from trl import SFTTrainer
from datasets import load_dataset

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [8]:
#The model for training
model_name='NousResearch/Llama-2-7b-chat-hf'

#Dataset for fine tune the data
dataset_name='mlabonne/guanaco-llama2-1k'

#Model after the finetuning
new_model_name='Llama2-7b-chat-finetune'

#QLoRA Parameters
lora_r=64
lora_alpha=16
lora_dropout=0.1

In [9]:
#Bitsandbytes parameters
use_4bit  = True
bnb_4bit_compute_dtype = 'float16'
bnb_4bit_quant_type = 'nf4'
use_nested_quant = True